In [1]:
from transformers import AutoTokenizer, LlamaForCausalLM
import torch

/usr/scratch/haokang/anaconda3/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_ID = "LoftQ/Llama-2-7b-hf-fp16-64rank-gsm8k"
model = LlamaForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.bfloat16,  # you may change it with different models
        token=None,
        device_map="cuda:1",
        cache_dir="../cache",
        use_cache=True,
    )

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.70s/it]


In [4]:
import transformers
TOKEN_ID = "meta-llama/Llama-2-7b-hf"
tokenizer = transformers.AutoTokenizer.from_pretrained(
        TOKEN_ID,
        # model_args.model_name_or_path,
        token=None,
        model_max_length=512,
        padding_side="left",
        use_fast=False,
        cache_dir="../cache",
    )

tokenizer_config.json: 100%|██████████| 776/776 [00:00<00:00, 5.81MB/s]
tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 2.29MB/s]
special_tokens_map.json: 100%|██████████| 414/414 [00:00<00:00, 3.05MB/s]


In [8]:
from datasets import load_dataset
dataset = load_dataset("gsm8k","main", cache_dir="../cache")
test_set = dataset["test"]

In [9]:
import math
QUESTION_PROMPT = "\nAnswer the above question. First think step by step and then answer the final number.\n"
question = [f"{example['question']}{QUESTION_PROMPT}" for example in test_set]
answer = []

# get numerical answer
for example in test_set["answer"]:
    ans = example.split("####")[-1]
    ans = ans.replace(",", "")  # handle numbers like 2,000
    try:
        ans = float(ans)
    except ValueError:
        ans = float("inf")
    answer.append(ans)
eval_step = math.ceil(len(question) / 1)

In [10]:
question_data = []
for i in range(eval_step):
    if i < eval_step - 1:
        batch = tokenizer(
            question[i * 1 : (i + 1) * 1],
            return_tensors="pt",
            padding="longest",
        )
    else:
        batch = tokenizer(
            question[i * 1 :],
            return_tensors="pt",
            padding="longest",
        )
    batch["input_len"] = len(batch["input_ids"][0])
    question_data.append(batch)

model.eval()
gen_kwargs = {
    "max_new_tokens": 256,
    "temperature": 0.8,
    "top_k": 40,
    "top_p": 0.95,
    "do_sample": True,
    "use_cache": True,
}
ans_pred_list = []

Using pad_token, but it is not set yet.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

: 

In [ ]:
for step, batch in enumerate(question_data):
    with torch.no_grad():
        gen_kwargs["input_ids"] = batch["input_ids"].to(0)
        gen_kwargs["attention_mask"] = batch["attention_mask"].to(0)
        generated_tokens = model.generate(**gen_kwargs)
        torch.cuda.synchronize()

    pred_tokens = generated_tokens[:, batch["input_len"] :]
    decoded_pred = tokenizer.batch_decode(pred_tokens, skip_special_tokens=True)
    break